In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from itertools import combinations
import os
import gc

In [3]:
labels = []
elist = []
tlist = []
slist = []
for year in range(2004,2012):
    print(year)
    df = pd.read_csv(f"../RawData/DAWN/DAWN-{year}-DS0001-data-excel.tsv", sep="\t")
    for idx, row in df.iterrows():
        drug_cols = [i for i in row.index if i.startswith("DRUGID") and row[i] != -7]
        t = row['YEAR'] + (row['QUARTER']-1) / 4
        drugs = list(row[drug_cols])
        for i,j in combinations(drugs, 2):
            i, j = np.sort([i,j])
            elist.append((i, j, t))

        for i,j, k in combinations(drugs, 3):
            i, j,k = np.sort([i,j, k])
            tlist.append((i, j, k, t))

        for col in drug_cols:
            idx = int(col.split("_")[1])
            labels.append((row[col], row[f'CATID_1_{idx}'], row[f'CATID_2_{idx}'], row[f'CATID_3_{idx}']))

        slist.append(drugs + [t])
    gc.collect()

2004
2005
2006
2007
2008
2009


/state/partition1/llgrid/pkg/anaconda/anaconda3-2021a/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,13,280,281,282,283,284) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2010
2011


In [4]:
label_df = pd.DataFrame(labels)
label_df.columns = ['id', 'group_id1', 'group_id2', 'group_id3']
label_df['group'] = label_df[['group_id1', 'group_id2', 'group_id3']].apply(tuple, axis=1)

In [5]:
all_labels = [ i for i in pd.unique(label_df['group'])]
label_dict = {name: i for i, name in enumerate(all_labels)}
label_df['group_code'] = label_df['group'].map(label_dict) 
label_df = label_df[['id', 'group_code']].drop_duplicates()

In [7]:
dir_name = "hosp-DAWN"
os.makedirs(f"../Data/{dir_name}", exist_ok=True)

label_df.to_csv(f'../Data/{dir_name}/labels.csv', index=False)

e_df = pd.DataFrame(elist)
e_df.columns = ['node_1', 'node_2', 't']
e_df = e_df[['node_1', 'node_2']].drop_duplicates()
e_df.to_csv(f'../Data/{dir_name}/edges.csv', index=False)

t_df = pd.DataFrame(tlist)
t_df.columns = ['node_1', 'node_2', 'node_3', 't']
t_df = t_df[['node_1', 'node_2', 'node_3']].drop_duplicates()
t_df.to_csv(f'../Data/{dir_name}/triangles.csv', index=False)

with open(f'../Data/{dir_name}/simplices.csv', 'w') as f:
    for item in slist:
        f.write("%s\n" % ",".join([str(i) for i in item]))